In [147]:
import pandas as pd
pd.set_option('display.max_columns', 300)

import numpy as np
import pickle
from pickle import load
from sklearn.linear_model import LinearRegression
import pandas as pd
import pickle
import numpy as np
import joblib
# pd.options.display.float_format = '{:,.0f}'.format 

## Step 1: Read in hold out data, scalers, and best model

In [148]:
pickle_in = open("model.pickle","rb")
holdout = pd.read_csv('./data/kc_house_data_test_features.csv', index_col=0)

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [149]:
holdout.columns

Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [150]:
holdout[holdout['bedrooms'] == 33] = holdout[holdout['bedrooms'] == 3]
holdout['bedrooms'] = holdout.bedrooms.apply(lambda x: 1 if x < 1 else x)
holdout['bathrooms'] = holdout.bathrooms.apply(lambda x: 1 if x < 1 else x)

In [151]:
df = holdout.copy()
df.head()
df_zip_dummies = pd.get_dummies(holdout['zipcode'],drop_first=True)
holdout2 = pd.concat([df, df_zip_dummies], axis=1)
holdout2.drop(columns=['id','date','waterfront','floors','sqft_lot15'],inplace=True)
holdout2.shape

(4323, 84)

In [144]:
transformed_holdout = holdout2.copy()
transformed_holdout.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,4,2,2270,11500,0,3,8,1540,730,1967,0,98034,48,-122,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4,2,2270,11500,0,3,8,1540,730,1967,0,98034,48,-122,2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2,1470,1779,0,3,8,1160,310,2005,0,98029,48,-122,1470,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,2,1280,16200,0,3,8,1030,250,1976,0,98077,48,-122,1160,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,3,2830,8126,0,3,8,2830,0,2005,0,98059,47,-122,2830,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [145]:
transformed_holdout.columns

Index([     'bedrooms',     'bathrooms',   'sqft_living',      'sqft_lot',
                'view',     'condition',         'grade',    'sqft_above',
       'sqft_basement',      'yr_built',  'yr_renovated',       'zipcode',
                 'lat',          'long', 'sqft_living15',           98002,
                 98003,           98004,           98005,           98006,
                 98007,           98008,           98010,           98011,
                 98014,           98019,           98022,           98023,
                 98024,           98027,           98028,           98029,
                 98030,           98031,           98032,           98033,
                 98034,           98038,           98039,           98040,
                 98042,           98045,           98052,           98053,
                 98055,           98056,           98058,           98059,
                 98065,           98070,           98072,           98074,
                 98075,  

## Step 3: Predict the holdout set

In [155]:
joblib_file = "model.pkl"  
final_model = joblib.load(joblib_file)
final_answers = final_model.predict(transformed_holdout)
final_answers

array([2.66696630e+10, 2.66696630e+10, 1.98180109e+10, ...,
       1.95422882e+10, 2.02420484e+10, 1.93487373e+10])

## Step 4: Export your predictions

In [156]:
final_answer = pd.DataFrame(final_predictions)
final_answer.to_csv('housing_preds_ismael_araujo.csv')